# Set up the Edgar Postgres DB

### Set up imports

In [15]:
import os

import psycopg2

### Set up database connection

Make sure the correct variables exist in your environment.

In [16]:
# Load credentials from environment. 
POSTGRES_ADDRESS = os.environ['POSTGRES_ADDRESS']
POSTGRES_PORT = os.environ['POSTGRES_PORT']
POSTGRES_USERNAME = os.environ['POSTGRES_USERNAME']
POSTGRES_PASSWORD = os.environ['POSTGRES_PASSWORD']
POSTGRES_DBNAME = os.environ['POSTGRES_DBNAME']

In [17]:
# Create connection and cursor    
conn = psycopg2.connect(host=POSTGRES_ADDRESS,
                  database=POSTGRES_DBNAME,
                  user=POSTGRES_USERNAME,
                  password=POSTGRES_PASSWORD,
                  port=POSTGRES_PORT)
cur = conn.cursor()

### Create the income table

(Only run this if it doesn't already exist)

In [32]:
# Create the Income Statement table
# fillingDate is purposefully misspelled to match the JSON response format. 
cur.execute("""CREATE TABLE income
                (symbol varchar(5),
                fillingDate timestamp,
                period varchar(5),
                revenue bigint, 
                costOfRevenue bigint, 
                grossProfit bigint, 
                grossProfitRatio float, 
                researchAndDevelopmentExpenses bigint, 
                generalAndAdministrativeExpenses bigint, 
                sellingAndMarketingExpenses bigint,
                otherExpenses bigint, 
                operatingExpenses bigint, 
                costAndExpenses bigint, 
                interestExpense bigint, 
                depreciationAndAmortization bigint, 
                ebitda bigint, 
                ebitdaratio float, 
                operatingIncome bigint, 
                operatingIncomeRatio float,
                totalOtherIncomeExpensesNet bigint, 
                incomeBeforeTax bigint, 
                incomeBeforeTaxRatio float, 
                incomeTaxExpense bigint, 
                netIncome bigint, 
                netIncomeRatio float, 
                eps float, 
                epsdiluted float, 
                weightedAverageShsOut bigint, 
                weightedAverageShsOutDil bigint);""") 

# Commit the table creation transaction.
conn.commit()

### Display contents of the income table

Feel free to run this interactively to debug table population.

In [14]:
query = """SELECT * FROM income;"""
cur.execute(query)
cur.fetchall()

[('AAPL',
  datetime.datetime(2020, 10, 30, 0, 0),
  'FY',
  274515000000,
  169559000000,
  104956000000,
  0.38233247727810865,
  18752000000,
  19916000000,
  0,
  -87000000,
  38668000000,
  208227000000,
  2873000000,
  11056000000,
  77344000000,
  0.2817478097736007,
  66288000000,
  0.24147314354406862,
  -87000000,
  67091000000,
  0.24439830246070343,
  9680000000,
  57411000000,
  0.20913611278072236,
  3.31,
  3.28,
  16935119000,
  17113688000),
 ('AAPL',
  datetime.datetime(2019, 10, 31, 0, 0),
  'FY',
  260174000000,
  161782000000,
  98392000000,
  0.3781776810903472,
  16217000000,
  18245000000,
  0,
  422000000,
  34462000000,
  196244000000,
  3576000000,
  12547000000,
  76477000000,
  0.29394559025882677,
  63930000000,
  0.24572017188496928,
  422000000,
  65737000000,
  0.2526655238417367,
  10481000000,
  55256000000,
  0.21238094505984456,
  2.9925,
  2.9725,
  18471336000,
  18595652000),
 ('AAPL',
  datetime.datetime(2018, 11, 5, 0, 0),
  'FY',
  26559500000

In [41]:
conn.commit()
query = """
SELECT symbol, revenue
FROM 
    (
    SELECT symbol, revenue
    FROM income 
    ORDER BY revenue desc
    ) AS foo
WHERE rownum <= 3
ORDER BY revenue;
"""
query = """
select ebitda, symbol from income
order by ebitda desc
limit 5
"""
cur.execute(query)
cur.fetchall()

[('BRK-B',), ('AAPL',), ('AAPL',), ('AAPL',), ('AAPL',)]

In [52]:
conn.commit()
query = """
SELECT SUM(revenue) FROM income WHERE symbol = 'AAPL' AND fillingDate >= now() - interval '2 years'
"""
cur.execute(query)
x = cur.fetchall()

In [55]:
type(x[0][0])


decimal.Decimal

In [13]:
query = """
ALTER TABLE income
ALTER COLUMN fillingDate TYPE timestamp USING fillingdate::timestamp without time zone;
"""
cur.execute(query)
conn.commit()

In [30]:
# Commit the transaction.
conn.commit()

In [36]:
# To delete all rows, run """DELETE FROM income;"""
# To delete the table, run """DROP TABLE income;"""